In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
# Read the excel file
DATA= pd.read_excel('./2020_BBGA_0303.xlsx')

In [41]:
# Data cleaning - preparing the dataset
data = DATA[["sdnaam",
             "gebiedcode15",
             "gebiedcodenaam",
             "jaar",
             "BEVDICHT",
             "WHUUR_GEM",
             "VKPARKEREN_R",
             "VKFIETSPARKEREN_R",
             "VVEILIGDAG_R",
             "VVEILIGAVOND_R",
             "VCRIMINALITEIT_R",
             "LHORECAOVERLAST_R",
             "BHSTART_P",
             "BHHORECA_1000INW",
             "SRCULTVOORZ_R",
             "SRSPORTGELEGENHEDEN_R",
             "WZZORGVOORZIENINGEN_R",
             "LOUD_R",
             "LKIND_R",
             "OAANBODBAO_R",
             "LOMGANGGROEPENB_R"]]

# Renaming the variables
data = data.rename(columns = {"sdnaam" : "region",
                   "gebiedcode15" : "neigh_code",
                   "gebiedcodenaam" : "neigh",
                   "jaar" : "year",
                   "BEVDICHT" : "population_density",
                   "WZZORGVOORZIENINGEN_R" : "care_facilities",
                   "SRCULTVOORZ_R" : "cultural_facilities",
                   "LOMGANGGROEPENB_R" : "group_interaction",
                   "OAANBODBAO_R" : "primary_schools",
                   "LHORECAOVERLAST_R" : "nuisance_hospitality",
                   "BHHORECA_1000INW" : "hospitality_per_1000",
                   "WHUUR_GEM" : "avg_rent",
                   "SRSPORTGELEGENHEDEN_R" : "sport_facilities",
                   "VVEILIGAVOND_R" : "safe_night",
                   "VVEILIGDAG_R" : "safe_day",
                   "VCRIMINALITEIT_R" : "nuisance_crime",
                   "VKFIETSPARKEREN_R"  : "parking_facilities_bike",
                   "VKPARKEREN_R"  : "parking_facilities",
                   "LKIND_R" : "child_friendly",
                   "LOUD_R": "suitable_growing_old",
                   "BHSTART_P":"start-ups"})
# Should be reversed:
# avg_rent - the lower the rent, the better
# psycho_distress - the lower the percentage, the better
# lonely - the lower the percentage, the better


# List of the variables
list_var=["population_density",
          "avg_rent",
          "parking_facilities",
          "parking_facilities_bike",
          "safe_day",
          "safe_night",
          "nuisance_crime",
          "nuisance_hospitality",
          "start-ups",
          "hospitality_per_1000",
          "cultural_facilities",
          "sport_facilities",
          "care_facilities",
          "suitable_growing_old",
          "child_friendly",
          "primary_schools",
          "group_interaction"]


# rename regions
new_names={"A Centrum":"centrum", "E West":"west", "F Nieuw-West":"n_west", "K Zuid":"zuid",
           "M Oost":"oost", "N Noord":"noord", "T Zuidoost":"zuidoost", "B Westpoort":"westpoort"}
data=data.replace({"region":new_names})
list_reg=["centrum","west","n_west","zuid","oost","noord","zuidoost","westpoort"] ###

# list_reg=new_names.values()
# list_reg=["centrum","west","n_west","zuid","oost","noord","zuidoost","westpoort"]

# Change the regions neighbourhood code
new_codes={"A":"centrum", "E":"west", "F":"n_west", "K":"zuid",
           "M":"oost", "N":"noord", "T":"zuidoost", "B":"westpoort"}
data=data.replace({"neigh_code":new_codes})

# Select year 2019 (no data in 2020)
year_2019 = data[data["year"]==2019].index
data = data.iloc[year_2019,:]

# Inspect null values
null_cols = data.isnull().sum()
null_cols

# --- Region has a lot of null, delete
region_null = data[data["region"].isnull()].index
data=data.drop(region_null)

# Select only the variables that have available rent information
rent_null = data[data["avg_rent"].isnull()].index
data=data.drop(rent_null)

# # Drop all empty variables --- I DID NOT INCLUDE THESE IN THE DATA SELECTION
# data=data.drop(["tourists_density", "free_parking", "div_gay", "div_discr", "social",
#                "bike_parking"], axis=1)

# Reset index
data=data.reset_index(drop=True)
data = data.drop([0]) # drop the STAD AMSTERDAM
data=data.reset_index(drop=True)


# Check the table
#data.head(10)
#data.isnull().sum()


In [42]:
# Functions to initialize scoring tables

def create_score_table_n(region):
    # Input: a region
    # Output: a scoring table for all the neighs of the specific region
    
    neigh_index=data[(data["region"]==region)&
        (data["region"]!=data["neigh_code"])].index
    region_table=data.iloc[neigh_index,:]
    region_table=region_table[["region", "neigh_code", "neigh","year"]].reset_index()
    return region_table

def create_score_table_r():
    # Output: a scoring table for all regions
    neigh_index=data[(data["region"]==data["neigh_code"])].index
    region_table=data.iloc[neigh_index,:]
    region_table=region_table[["region", "neigh_code", "neigh","year"]].reset_index()
    return region_table

#create_score_table_n("west")

In [43]:
# Functions to compare a variable between several regions or neighbourhoods
# Input: formated table (data)

def new_neigh_winner(variable, region):
    # Example of how to use: neigh_winner("horeca_estab", "west")
    # Input: name of the variable and region
    # Output: a series of the score of each neigh for specific variable
    
    # Define some necessary variables to be used later, import the indexes of the
    # create_score_table()
    region_table=pd.DataFrame
    score_list=[]
    score_df=pd.DataFrame()
    
    # Make a small table for the region selected
    # --- Step 1: isolate all neighs of one region
    neigh_index=data[(data["region"]==region)&
        (data["region"]!=data["neigh_code"])].index
    region_table=data.iloc[neigh_index,:]
    # --- Step 2: throw away rest of variables
    region_table=region_table[["region", "neigh_code", "neigh","year",variable]]
    
    # Iterate and calculate scores based on the max value
    for i in region_table[variable]:
        # create a list of the values and convert to series
        MAX=max(region_table[variable])
        score_list.append(i/MAX)
        score_df=pd.DataFrame(score_list, columns=[variable])
    return (score_df)


def new_region_winner(variable):
    # Example of how to use: neigh_winner("horeca_estab")
    # Input: name of the variable
    # Output: a series of the score of each region for specific variable
    
    # Define some necessary variables to be used later, import the indexes of the
    # create_score_table()
    region_table=pd.DataFrame
    score_list=[]
    score_df=pd.DataFrame()
    
    # Make a small table for the region selected
    # --- Step 1: isolate all neighs of one region
    neigh_index=data[(data["region"]==data["neigh_code"])].index
    region_table=data.iloc[neigh_index,:]
    # --- Step 2: throw away rest of variables
    region_table=region_table[["region", "neigh_code", "neigh","year",variable]]
    
    # Iterate and calculate scores based on the max value
    for i in region_table[variable]:
        # create a list of the values and convert to series
        MAX=max(region_table[variable])
        score_list.append(i/MAX)
        score_df=pd.DataFrame(score_list, columns=[variable])
    return (score_df)

# Test the functions
#new_neigh_winner("horeca_estab", "west")
#new_region_winner("horeca_estab")

In [44]:
# --- Create MASTER_SCORE_NEIGH
# ------ table with all regions & neighs with their score (score based on each region)

# Define 2 dicts will be used to teporarily store values while iterating 
reg={}
var={}

# First loop will iterate through regions, preparing a scoring table with neigh names, year etc
# and assign it to a dictionary object. The key is the region (eg "WEST") and value the scoring
# table as dataframe
for region in list_reg:
    reg[region.upper()] = create_score_table_n(region)
    # Second loop iterates through variables, calculating the score for each neigh and concatinating
    # the series of the neigh values for each variable (produced in the loop) with the scoring table
    # (again creating an intermediate dict "var" - like "reg" in previous loop)
    for variable in list_var:
        var[variable]=new_neigh_winner(variable,region)
        reg[region.upper()]=pd.concat([reg[region.upper()],var[variable]],axis=1).reset_index(drop=True)
        # product of this nested loop will be a dict "reg" with regions as keys (eg "WEST", "ZUID", etc)
        # and dataframes of the scores as values.

# Finally, creating a MASTER_SCORE_NEIGH table for all the neighs
MASTER_SCORE_NEIGH=pd.DataFrame()
for i in reg.keys():
    MASTER_SCORE_NEIGH=pd.concat([MASTER_SCORE_NEIGH,reg[i]], axis=0).reset_index(drop=True)

# Drop additional index column    
MASTER_SCORE_NEIGH=MASTER_SCORE_NEIGH.drop(["index"], axis=1)

# Reverse score for rent
# The lower the rent the better for the residents.
MASTER_SCORE_NEIGH["avg_rent"] = 1-MASTER_SCORE_NEIGH["avg_rent"]

# Add a column that sums the score of all variables. To do that we create a dummy in order to not drop the
# year column in the MASTER file. (year is the only numerical)
dummy=MASTER_SCORE_NEIGH
dummy=dummy.drop(["year"], axis=1)
sum_=dummy.sum(axis=1)
sum_=sum_.rename("SUM")
MASTER_SCORE_NEIGH=pd.concat([MASTER_SCORE_NEIGH,sum_],axis=1)
MASTER_SCORE_NEIGH

# Order based on SUM
MASTER_SCORE_NEIGH=MASTER_SCORE_NEIGH.sort_values(by="SUM", ascending=False)

# Test
MASTER_SCORE_NEIGH
# MASTER_SCORE_NEIGH.shape

,region,neigh_code,neigh,year,population_density,avg_rent,parking_facilities,parking_facilities_bike,safe_day,safe_night,...,start-ups,hospitality_per_1000,cultural_facilities,sport_facilities,care_facilities,suitable_growing_old,child_friendly,primary_schools,group_interaction,SUM
118,zuidoost,T97,T97 Gein,2019.0,0.769767,0.000000,0.985507,1.000000,1.000000,1.000000,...,1.000000,0.166667,0.903226,0.985294,0.971831,1.000000,1.000000,1.000000,1.000000,14.782292
117,zuidoost,T96,T96 Holendrecht/Reigersbos,2019.0,0.865116,0.053068,1.000000,1.000000,0.950617,0.929577,...,0.971098,0.333333,0.919355,1.000000,1.000000,0.958333,0.894737,0.986667,0.971831,14.727520
19,west,E18,E18 Kinkerbuurt,2019.0,0.918331,0.168847,0.882353,0.866667,0.988636,0.962963,...,0.666667,0.928571,0.950000,0.901408,0.972973,0.905405,0.818182,0.974026,0.972973,14.677189
113,zuidoost,DX21,DX21 Bijlmer-Oost,2019.0,0.973562,0.008292,0.927536,0.919355,0.975309,0.957746,...,0.965318,0.333333,0.983871,1.000000,0.957746,0.944444,0.881579,0.960000,0.985915,14.667794
120,zuidoost,DY26,DY26 Bijlmer-Oost,2019.0,0.973562,0.008292,0.927536,0.919355,0.975309,0.957746,...,0.965318,0.333333,0.983871,1.000000,0.957746,0.944444,0.881579,0.960000,0.985915,14.667794
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,centrum,A00,A00 Burgwallen-Oude Zijde,2019.0,0.528028,0.160215,0.671642,0.737705,0.943182,0.911392,...,0.776596,0.943820,0.941176,0.718750,0.857143,0.794521,0.557143,0.776316,0.878378,12.686998
90,oost,M57,M57 Betondorp,2019.0,0.113701,0.401776,0.986301,0.803030,0.965909,0.938272,...,0.476471,0.375000,0.643836,0.853659,0.763158,0.987013,0.938272,0.615385,0.907895,12.682583
3,centrum,A01,A01 Burgwallen-Nieuwe Zijde,2019.0,0.315720,0.000000,0.776119,0.836066,0.954545,0.873418,...,0.819149,1.000000,0.952941,0.750000,0.857143,0.712329,0.457143,0.710526,0.864865,12.397288
86,oost,M34,M34 Zeeburgereiland/Nieuwe diep,2019.0,0.056517,0.209767,0.794521,0.893939,0.954545,0.864198,...,1.000000,0.625000,0.479452,0.658537,0.644737,0.792208,0.827160,0.833333,0.934211,12.323137


In [45]:
# --- Create MASTER_SCORE_REG
# ------ table with all regions with their score

# Define 1 dict that will be used to teporarily store values while iterating 
var={}

# Preparing a scoring table with region names, year etc as dataframe
reg = create_score_table_r()

#"for" loop iterates through variables, calculating the score for each region and concatinating
# the series of the region values for each variable with the scoring table
# Creates a temp dictionary where the key is the variable name and value the scoring table as series
for variable in list_var:
    var[variable]=new_region_winner(variable)
    reg=pd.concat([reg,var[variable]],axis=1).reset_index(drop=True)

# Finally, creating a MASTER_SCORE_REG table for all the neighs
MASTER_SCORE_REG=pd.DataFrame()
for i in reg.keys():
    MASTER_SCORE_REG=pd.concat([MASTER_SCORE_REG,reg[i]], axis=1).reset_index(drop=True)
# Drop additional index column    
MASTER_SCORE_REG=MASTER_SCORE_REG.drop(["index"], axis=1)

# Reverse score for rent
# The lower the rent the better for the residents.
MASTER_SCORE_REG["avg_rent"] = 1-MASTER_SCORE_REG["avg_rent"]

# Add a column that sums the score of all variables. To do that we create a dummy in order to not drop the
# year column in the MASTER file. (year is the only numerical)
dummy=MASTER_SCORE_REG
dummy=dummy.drop(["year"], axis=1)
sum_=dummy.sum(axis=1)
sum_=sum_.rename("SUM")
MASTER_SCORE_REG=pd.concat([MASTER_SCORE_REG,sum_],axis=1)
MASTER_SCORE_REG

# Order based on SUM
MASTER_SCORE_REG=MASTER_SCORE_REG.sort_values(by="SUM", ascending=False)

# Test
MASTER_SCORE_REG

,region,neigh_code,neigh,year,population_density,avg_rent,parking_facilities,parking_facilities_bike,safe_day,safe_night,...,start-ups,hospitality_per_1000,cultural_facilities,sport_facilities,care_facilities,suitable_growing_old,child_friendly,primary_schools,group_interaction,SUM
0,centrum,centrum,A Centrum,2019.0,0.901010,0.028571,0.863636,0.881356,1.000000,0.974684,...,0.573427,1.000000,1.000000,0.884058,0.957143,0.945205,0.857143,0.960000,1.000000,14.598599
1,west,west,E West,2019.0,1.000000,0.162733,0.924242,0.898305,0.976744,0.949367,...,0.734266,0.291667,0.871795,0.956522,0.985714,0.904110,0.914286,0.973333,0.972222,14.386437
3,zuid,zuid,K Zuid,2019.0,0.614204,0.000000,1.000000,1.000000,1.000000,1.000000,...,0.636364,0.291667,0.871795,1.000000,1.000000,1.000000,0.985714,1.000000,1.000000,14.338015
4,oost,oost,M Oost,2019.0,0.514826,0.120497,1.000000,1.000000,1.000000,0.949367,...,0.692308,0.208333,0.794872,1.000000,0.971429,0.972603,1.000000,0.973333,1.000000,14.118081
6,zuidoost,zuidoost,T Zuidoost,2019.0,0.289978,0.296894,0.969697,0.983051,0.918605,0.848101,...,1.000000,0.125000,0.743590,0.971014,0.957143,0.904110,0.957143,0.973333,0.972222,13.786594
5,noord,noord,N Noord,2019.0,0.152467,0.238509,0.984848,0.932203,0.930233,0.873418,...,0.769231,0.166667,0.705128,0.913043,0.928571,0.972603,1.000000,0.933333,0.958333,13.322955
2,n_west,n_west,F Nieuw-West,2019.0,0.306552,0.181366,0.939394,0.949153,0.895349,0.810127,...,1.000000,0.125000,0.730769,0.942029,0.914286,0.890411,0.900000,0.933333,0.916667,13.180926


In [47]:
# --- Create dataframe with all variable values translated into bins

# Neighbourhoods
labels = ["Low","Moderate","High"]
score_neigh_bins = MASTER_SCORE_NEIGH.copy()

for variable in list_var:
    bins = pd.cut(score_neigh_bins[variable],3, labels = labels)
    score_neigh_bins[variable] = bins

# Regions
labels = ["Low","Moderate","High"]
score_reg_bins = MASTER_SCORE_REG.copy()

for variable in list_var:
    bins = pd.cut(score_reg_bins[variable],3, labels = labels)
    score_reg_bins[variable] = bins

In [48]:
# # Export to excel
# MASTER_SCORE_REG.to_excel('./MASTER_SCORE_REG.xlsx', index=False)
# MASTER_SCORE_NEIGH.to_excel('./MASTER_SCORE_NEIGH.xlsx', index=False)

In [49]:
# Calculate weights


In [50]:
MASTER_SCORE_REG.columns

Index(['region', 'neigh_code', 'neigh', 'year', 'population_density',
       'avg_rent', 'parking_facilities', 'parking_facilities_bike', 'safe_day',
       'safe_night', 'nuisance_crime', 'nuisance_hospitality', 'start-ups',
       'hospitality_per_1000', 'cultural_facilities', 'sport_facilities',
       'care_facilities', 'suitable_growing_old', 'child_friendly',
       'primary_schools', 'group_interaction', 'SUM'],
      dtype='object')

In [115]:
# Calculate health region
health = ['care_facilities', 'sport_facilities','suitable_growing_old','child_friendly','primary_schools']
drop_col = ['year','population_density','avg_rent','parking_facilities','parking_facilities_bike','safe_day',
       'safe_night','nuisance_crime','nuisance_hospitality','start-ups',
       'hospitality_per_1000','cultural_facilities','group_interaction','SUM']
HEALTH_SCORE_REG = MASTER_SCORE_REG.copy()
HEALTH_SCORE_REG = HEALTH_SCORE_REG.drop(drop_col, axis = 1)

health_sum = HEALTH_SCORE_REG.sum(axis=1)
health_sum = health_sum.rename('health_sum')
HEALTH_SCORE_REG = pd.concat([HEALTH_SCORE_REG, health_sum],axis=1)

HEALTH_SCORE_REG

,region,neigh_code,neigh,sport_facilities,care_facilities,suitable_growing_old,child_friendly,primary_schools,health_sum
0,centrum,centrum,A Centrum,0.884058,0.957143,0.945205,0.857143,0.960000,4.603549
1,west,west,E West,0.956522,0.985714,0.904110,0.914286,0.973333,4.733965
3,zuid,zuid,K Zuid,1.000000,1.000000,1.000000,0.985714,1.000000,4.985714
4,oost,oost,M Oost,1.000000,0.971429,0.972603,1.000000,0.973333,4.917365
6,zuidoost,zuidoost,T Zuidoost,0.971014,0.957143,0.904110,0.957143,0.973333,4.762743
5,noord,noord,N Noord,0.913043,0.928571,0.972603,1.000000,0.933333,4.747551
2,n_west,n_west,F Nieuw-West,0.942029,0.914286,0.890411,0.900000,0.933333,4.580059


In [116]:
# Calculate comfort region
comfort = ['avg_rent','population_density','parking_facilities','parking_facilities_bike','start']
drop_col = ['year','parking_facilities_bike','safe_day','safe_night','nuisance_crime', 
            'nuisance_hospitality','hospitality_per_1000','cultural_facilities', 
            'sport_facilities','care_facilities','suitable_growing_old', 
            'child_friendly','primary_schools','group_interaction','SUM']
COMFORT_SCORE_REG = MASTER_SCORE_REG.copy()
COMFORT_SCORE_REG = COMFORT_SCORE_REG.drop(drop_col, axis = 1)

comfort_sum = COMFORT_SCORE_REG.sum(axis=1)
comfort_sum = comfort_sum.rename('comfort_sum')
COMFORT_SCORE_REG = pd.concat([COMFORT_SCORE_REG, comfort_sum],axis=1)

COMFORT_SCORE_REG

,region,neigh_code,neigh,population_density,avg_rent,parking_facilities,start-ups,comfort_sum
0,centrum,centrum,A Centrum,0.901010,0.028571,0.863636,0.573427,2.366644
1,west,west,E West,1.000000,0.162733,0.924242,0.734266,2.821241
3,zuid,zuid,K Zuid,0.614204,0.000000,1.000000,0.636364,2.250568
4,oost,oost,M Oost,0.514826,0.120497,1.000000,0.692308,2.327630
6,zuidoost,zuidoost,T Zuidoost,0.289978,0.296894,0.969697,1.000000,2.556569
5,noord,noord,N Noord,0.152467,0.238509,0.984848,0.769231,2.145055
2,n_west,n_west,F Nieuw-West,0.306552,0.181366,0.939394,1.000000,2.427312


In [117]:
# Calculate happiness region
happiness = ['safe_night', 'safe_day','nuisance_crime','nuisance_hospitality','hospitality_per_1000','cultural_facilities','group_interaction']
drop_col = ['year','population_density','avg_rent', 'parking_facilities', 'parking_facilities_bike',
            'start-ups','care_facilities', 'suitable_growing_old', 'child_friendly','primary_schools',
            'SUM']
HAPPY_SCORE_REG = MASTER_SCORE_REG.copy()
HAPPY_SCORE_REG = HAPPY_SCORE_REG.drop(drop_col, axis = 1)

happiness_sum = HAPPY_SCORE_REG.sum(axis=1)
happiness_sum = happiness_sum.rename('happy_sum')
HAPPY_SCORE_REG = pd.concat([HAPPY_SCORE_REG, happiness_sum],axis=1)

HAPPY_SCORE_REG

,region,neigh_code,neigh,safe_day,safe_night,nuisance_crime,nuisance_hospitality,hospitality_per_1000,cultural_facilities,sport_facilities,group_interaction,happy_sum
0,centrum,centrum,A Centrum,1.000000,0.974684,0.945205,0.827160,1.000000,1.000000,0.884058,1.000000,7.631107
1,west,west,E West,0.976744,0.949367,0.945205,0.925926,0.291667,0.871795,0.956522,0.972222,6.889448
3,zuid,zuid,K Zuid,1.000000,1.000000,1.000000,0.938272,0.291667,0.871795,1.000000,1.000000,7.101733
4,oost,oost,M Oost,1.000000,0.949367,0.945205,0.975309,0.208333,0.794872,1.000000,1.000000,6.873086
6,zuidoost,zuidoost,T Zuidoost,0.918605,0.848101,0.876712,1.000000,0.125000,0.743590,0.971014,0.972222,6.455245
5,noord,noord,N Noord,0.930233,0.873418,0.876712,0.987654,0.166667,0.705128,0.913043,0.958333,6.411189
2,n_west,n_west,F Nieuw-West,0.895349,0.810127,0.808219,0.938272,0.125000,0.730769,0.942029,0.916667,6.166431


In [118]:
# List all quality scores
QUALITY_SCORE = MASTER_SCORE_REG.copy()
drop_col = ['year','neigh','neigh_code','population_density',
       'avg_rent', 'parking_facilities', 'parking_facilities_bike', 'safe_day',
       'safe_night', 'nuisance_crime', 'nuisance_hospitality', 'start-ups',
       'hospitality_per_1000', 'cultural_facilities', 'sport_facilities',
       'care_facilities', 'suitable_growing_old', 'child_friendly',
       'primary_schools', 'group_interaction']
QUALITY_SCORE = QUALITY_SCORE.drop(drop_col, axis = 1)
QUALITY_SCORE = pd.concat([QUALITY_SCORE,health_sum,comfort_sum,happiness_sum],axis=1)
QUALITY_SCORE

,region,SUM,health_sum,comfort_sum,happy_sum
0,centrum,14.598599,4.603549,2.366644,7.631107
1,west,14.386437,4.733965,2.821241,6.889448
3,zuid,14.338015,4.985714,2.250568,7.101733
4,oost,14.118081,4.917365,2.327630,6.873086
6,zuidoost,13.786594,4.762743,2.556569,6.455245
5,noord,13.322955,4.747551,2.145055,6.411189
2,n_west,13.180926,4.580059,2.427312,6.166431


In [119]:
# Identify low variance
low_variance = []
SCORE_REG = MASTER_SCORE_REG.copy()

for col in SCORE_REG._get_numeric_data():
    minimum = min(SCORE_REG[col])
    maximum = max(SCORE_REG[col])
    difference = maximum - minimum
    if difference < .25:
        low_variance.append(col)
SCORE_REG = SCORE_REG.drop(low_variance, axis = 1)
SCORE_REG = SCORE_REG.drop('SUM', axis = 1)
SCORE_REG

,region,neigh_code,neigh,population_density,avg_rent,start-ups,hospitality_per_1000,cultural_facilities
0,centrum,centrum,A Centrum,0.901010,0.028571,0.573427,1.000000,1.000000
1,west,west,E West,1.000000,0.162733,0.734266,0.291667,0.871795
3,zuid,zuid,K Zuid,0.614204,0.000000,0.636364,0.291667,0.871795
4,oost,oost,M Oost,0.514826,0.120497,0.692308,0.208333,0.794872
6,zuidoost,zuidoost,T Zuidoost,0.289978,0.296894,1.000000,0.125000,0.743590
5,noord,noord,N Noord,0.152467,0.238509,0.769231,0.166667,0.705128
2,n_west,n_west,F Nieuw-West,0.306552,0.181366,1.000000,0.125000,0.730769


In [120]:
# Identify low variance
low_variance = []
SCORE_NEIGH = MASTER_SCORE_NEIGH.copy()

for col in SCORE_NEIGH._get_numeric_data():
    minimum = min(SCORE_NEIGH[col])
    maximum = max(SCORE_NEIGH[col])
    difference = maximum - minimum
    if difference < .5:
        low_variance.append(col)
SCORE_NEIGH = SCORE_NEIGH.drop(low_variance, axis = 1)
SCORE_NEIGH = SCORE_NEIGH.drop('SUM', axis = 1)

SCORE_NEIGH

,region,neigh_code,neigh,population_density,avg_rent,start-ups,hospitality_per_1000,cultural_facilities,child_friendly
118,zuidoost,T97,T97 Gein,0.769767,0.000000,1.000000,0.166667,0.903226,1.000000
117,zuidoost,T96,T96 Holendrecht/Reigersbos,0.865116,0.053068,0.971098,0.333333,0.919355,0.894737
19,west,E18,E18 Kinkerbuurt,0.918331,0.168847,0.666667,0.928571,0.950000,0.818182
113,zuidoost,DX21,DX21 Bijlmer-Oost,0.973562,0.008292,0.965318,0.333333,0.983871,0.881579
120,zuidoost,DY26,DY26 Bijlmer-Oost,0.973562,0.008292,0.965318,0.333333,0.983871,0.881579
...,...,...,...,...,...,...,...,...,...
2,centrum,A00,A00 Burgwallen-Oude Zijde,0.528028,0.160215,0.776596,0.943820,0.941176,0.557143
90,oost,M57,M57 Betondorp,0.113701,0.401776,0.476471,0.375000,0.643836,0.938272
3,centrum,A01,A01 Burgwallen-Nieuwe Zijde,0.315720,0.000000,0.819149,1.000000,0.952941,0.457143
86,oost,M34,M34 Zeeburgereiland/Nieuwe diep,0.056517,0.209767,1.000000,0.625000,0.479452,0.827160
